In [1]:
import chess
import chess.engine

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [2]:
from gen_old import *

In [3]:
class ChessDataset(Dataset):
    """Chess Positions Evaluation Dataset"""

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        position = np.array(self.df.iloc[idx, :-1])
        eval_ = np.array(self.df.iloc[idx, -1]/100)
        
        sample = {'position': torch.from_numpy(position), 'eval': torch.from_numpy(eval_)}

        return sample

In [4]:
dataset = ChessDataset(csv_file='data/stockfish_depth0e.csv')

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.fc1 = nn.Linear(513, 64)
        # self.conv1 = nn.Conv1d(64, 8, 3) #input, output channels, kernel_size
        # self.conv2 = nn.Conv2d(8, 8, 3)
        # self.conv3 = nn.Conv2d(8, 8, 3)
        # self.conv4 = nn.Conv2d(8, 8, 3)
        self.fc2 = nn.Linear(64, 32) 
        self.fc3 = nn.Linear(32,8)
        self.fc4 = nn.Linear(8, 1) 

    def forward(self, x):
        x = self.fc1(x)
        # x = self.conv1(x)
        # x = self.conv2(x)
        # x = self.conv3(x)
        # x = self.conv4(x)   
        x = torch.reshape(x, (-1,)) # flattens into 1d tensor
        x = self.fc2(x)
        x = self.fc3(F.relu(x))
        x = self.fc4(torch.sigmoid(x))
        #x = self.fc1(x)
        #x = self.fc2(x)

        return x

In [6]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)

net = Net()#.cuda()

# Load model
net.load_state_dict(torch.load('chess_ai0.pt'))
net.eval()

Net(
  (fc1): Linear(in_features=513, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=1, bias=True)
)

In [5]:
t = torch.tensor([[1., -1.], [1., -1.]])
t = t.cuda()
t.get_device()

0

# TRAINING DATA

In [ ]:
#torch.cuda.get_device_name(device)

In [ ]:
trainloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0) 
#num_workers = 0 doesn't use GPU. GPU seems to be a lot slower than CPU,.
#data set is prob too big or my laptop nvidia GPU is just trash

optimizer = optim.Adam(net.parameters(), lr = 0.0009) #lr = 0.001
criterion = nn.MSELoss() 

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['position'].reshape(1, 513).float()#.cuda()
        labels = data['eval'].float()#.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 400 == 399:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 400))
            running_loss = 0.0

print('Finished Training')

In [7]:
def compare_eval():
    board = generate_rand_board()
    #print(convert_board(board))
    torch_board = torch.from_numpy(convert_board(board).reshape(1, 513)).float()
    net_eval = net(torch_board)
    sf_eval = stockfish_evaluation(board)
    print(f"NN: {net_eval} Stockfish: {sf_eval/100}")
    

In [8]:
for _ in range(5):
    compare_eval()

NN: tensor([5.1656], grad_fn=<AddBackward0>) Stockfish: 6.4
NN: tensor([5.8066], grad_fn=<AddBackward0>) Stockfish: 0.26
NN: tensor([5.7132], grad_fn=<AddBackward0>) Stockfish: 0.51
NN: tensor([3.3727], grad_fn=<AddBackward0>) Stockfish: -7.17
NN: tensor([-7.9268], grad_fn=<AddBackward0>) Stockfish: 3.53


In [9]:
# Choosing Best Move with Minimax

In [10]:
def nn_prediction(position: chess.Board):
    board_rep = convert_board(position)
    torch_board = torch.from_numpy(board_rep.reshape(1, 513)).float()
    return net(torch_board)


In [35]:
# Minimax implementation 
def minimax(position, depth, maximizingPlayer, alpha = -1e9, beta = 1e9):
    #print(depth)
    if depth == 0 or position.is_game_over():
        return nn_prediction(position)
        
    if maximizingPlayer:
        value = -np.Inf
        for move in position.legal_moves:
            position.push(move)
            value = max(value, minimax(position, depth-1, False, alpha, beta))
            position.pop()
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value
    else:
        value = np.Inf
        for move in position.legal_moves:
            position.push(move)
            value = min(value, minimax(position, depth-1, True, alpha, beta))
            position.pop()
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value

In [41]:
def find_best_move(position, depth):
    
  best_evaluation = -np.Inf

  for move in position.legal_moves:
    position.push(move)
    nn_eval = minimax(position, depth - 1, False, -np.Inf, np.Inf)
    position.pop()
    
    if nn_eval > best_evaluation:
      best_evaluation = nn_eval
      best_move = move
  
  return best_move

In [43]:
def play_game():
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")    
    while not board.is_game_over():   
        nn_move = find_best_move(board,4) 
        
        board.push(nn_move)
        print(board)
        print(nn_move)
        moved = False
        while not moved:
            try:
                sqr = input('your move')
                if sqr == 'q' or sqr == 'quit':
                    return
                first_letter = sqr[0]
                if first_letter == 'n' or first_letter == 'b'
                or first_letter == 'r'1. e4 e5 2. Qh5 Nc6 3. d4 Nf6 4. Qh4 Nxd4
5. c3 Nc2+ 6. Kd1 Nxa1 7. Nf3 Be7 8. Bc4 d5
9. exd5 Nxd5 10. Bb5+ c6 11. Bxc6+ bxc6 12. Qg3 Nxc3+
13. Ke1 Qd1#
                or first_letter == 'q'
                or first_letter == 'k' :
                    sqr = first_letter.upper() + sqr[1:]
                board.push_san(sqr)
                moved = True
            except:
                moved = False


In [44]:
play_game()

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . Q
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B . K B N R
d1h5
r . b q k b n r
p p p p . p p p
. . n . . . . .
. . . . p . . Q
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B . K B N R
d2d4
r . b q k b . r
p p p p . p p p
. . n . . n . .
. . . . p . . .
. . . P P . . Q
. . . . . . . .
P P P . . P P P
R N B . K B N R
h5h4
r . b q k b . r
p p p p . p p p
. . . . . n . .
. . . . p . . .
. . . n P . . Q
. . P . . . . .
P P . . . P P P
R N B . K B N R
c2c3
r . b q k b . r
p p p p . p p p
. . . . . n . .
. . . . p . . .
. . . . P . . Q
. . P . . . . .
P P n . . P P P
R N B K . B N R
e1d1
r . b q k b . r
p p p p . p p p
. . . . . n . .
. . . . p . . .
. . . . P . . Q
. . P . . N . .
P P . . . P P P
n N B K . B . R
g1f3
r . b q k . . r
p p p p b p p p
. . . . . n . .
. . . . p . . .
. . B . P . . Q
. . P . . N . .
P P . . . P P P
n N B K . . . R
f1c4
r . b q k . . r
p p p . b p p p
. . . . . n . .
. . . P p . . .
. . B

In [ ]:
# memoization ? transposition of positions